In [30]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

In [40]:
folder = "../../Data/hpc_out/rwm_wip_2_6r7m/"
mol_type = "6r7m"
#folder = "../../Data/collected_simulation_results/rwm_op_2_6r7m/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

acceptance_rates = Vector{Float64}([])
min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            # if input["T"] != 2.5
            #     continue
            # end
            id = parse(Int, split(file, separator)[id_index])
            exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
            exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
            theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
            string = "$(id): $(input["T"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
            push!(acceptance_rates, output["αs"][end])
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println(e)
            println("Error in $file")
        end
    end
end

In [41]:
println("Minimal configuration id: $(argmin(min_Es)) | Average acceptance rate: $(sum(acceptance_rates) / length(acceptance_rates))")

Minimal configuration id: 37 | Average acceptance rate: 0.9775836619924991


In [42]:
for elem in evaluation_strings
    println(elem)
end



3: 3.4 | 0.9995237 | 8219.603607019644, 8219.578303594551, 8219.621745622682 | 235.492662669337
4: 3.4 | 0.9747451 | 8219.69028516332, 8198.797782870828, 8219.578304508 | 14.75251795254021
5: 3.4 | 0.99675125 | 8219.587236464533, 8202.824258385608, 8219.588088427872 | 31.86080833677934
6: 3.4 | 0.99946624 | 8219.659140324204, 8219.578303471539, 8219.664587631962 | 314.09292463183687
7: 3.4 | 0.91715705 | 8219.642556769597, 8188.597915229487, 8219.583194470028 | 24.092645771012297
8: 3.4 | 0.99928623 | 8219.59107570818, 8219.578303503933, 8219.578944625644 | 230.78656594731163
9: 3.4 | 0.9964249 | 8219.733840982793, 8204.861974794192, 8219.578304532879 | 44.7133543422176
10: 3.4 | 0.99948156 | 8219.65638060677, 8217.47320020398, 8219.578467313753 | 52.43310631866884
11: 3.4 | 0.9973545 | 8219.58924240258, 8208.482303444402, 8219.578319813443 | 34.438003815066764
12: 3.4 | 0.9580127 | 8219.837831343466, 8192.7473962773, 8219.57958126589 | 9.876769427763087
13: 3.4 | 0.9676032 | 8544.32

In [23]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(output["Es"])
        state = output["states"][mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
    catch e
        println(e)
        continue
    end
end

# Print Single Assembly Path

In [34]:
@load "../../Data/hpc_out/rwm_op_3_6r7m/5_rwm_op_3_6r7m.jld2" input output


2-element Vector{Symbol}:
 :input
 :output

In [35]:
input["template_centers"]

3×1206 Matrix{Float64}:
 -27.8796   -26.6674   -26.5597   …  -31.3903   -31.5191   -32.6934
   9.063      9.54921    9.03875       8.50467    8.25724    9.091
  -4.79258   -4.1493    -2.71995      -1.62113   -0.2168    -2.14403

In [36]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
